In [1]:
import PySimpleGUI as sg
import cv2
import numpy as np
import sys
from sys import exit as exit
from datetime import datetime
import requests
import pandas as pd
from tkinter import messagebox
import csv
import time
import asyncio
import aiohttp

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


# funcion para leer la placa
async def leer_placa(img):
    regions = ['gb', 'it']
    with open(img, 'rb') as fp:
        # se pide la consulta al servidor
        response = requests.post(
            'https://api.platerecognizer.com/v1/plate-reader/',
            data=dict(regions=regions),
            files=dict(upload=fp),
            headers={'Authorization': 'Token'})
    return response.json()


async def registrar_placa_en_sistema(numero_placa):
    try:
        data = {
            "carNumberPlate": numero_placa,
            "checkpointArrivalName": "Achica arriba 1",
            "checkpointExitName": "Achica arriba 2"
        }
        async with aiohttp.ClientSession() as session:
            async with session.post('http://localhost:8090/api/drivers/add-report-penalty', json=data, auth=aiohttp.BasicAuth('policia@sis', 'zsE&;3w}GZk;4x^')) as resp:
                contentType = resp.headers.get('content-type')
                if contentType == 'application/json':
                    response = await resp.json()
                    if 'carFeatures' in response:
                        if 'numberPlate' in response['carFeatures']:
                            print("Placa registrada en el sistema: ",
                                  response['carFeatures']['numberPlate'])
                    return response
                    return await resp.json()
                else:
                    return await resp.text()
    except Exception as e:
        print("Error al registrar placa en el sistema", e)
        return None


async def validar_placa(data, lista_placas, file, writer, max_num_plates, fechas):
    # siempre que la imagen sea una placa
    if data is not None and 'results' in data and data['results'] != []:
        # validar que no haya placas repetidas
        for result in data['results']:
            license_plate = result['plate'].upper()
        if len(lista_placas) > 1:  # para comenzar a verificar a partir de 1 placa
            if result['plate'] in lista_placas:
                validar = 1  # filtra placas repetidas
            else:
                lista_placas.append(license_plate)
                response = await registrar_placa_en_sistema(license_plate)
                validar = 0  # asigna cero si no esta repetida
        else:  # se agrega la primera placa
            lista_placas.append(license_plate)
            response = await registrar_placa_en_sistema(license_plate)
            validar = 0
        # verificar que haya menos de maximo numero de carros
        if len(lista_placas) > max_num_plates:
            validar = 2
            lista_placas.pop()  # elimina la ultima placa de la lista
        # se verifica si hay espacio en el parking o si esta repetida la placa antes de guardar
        guardar_placa(data, file, writer, validar)
        # Mensaje para saber que se leyo correctamente la placa
        # messagebox.showinfo(message='PLACA leída "CORRECTAMENTE"')
        # Se lee la fecha y la hora
        date_ = datetime.today().strftime('%x %X')  # leer le fecha y hora
        fechas.append(date_.split(' '))
        # date,time_=date_[0], date_[1]

    # else: # cuando la imagen no tenga una placa
    #    messagebox.showerror("ERROR", 'La imagen NO FUE RECONOCIDA')


# funcion para ir guardando en el archivo cada placa leida
def guardar_placa(data, file, writer, validar):
    for result in data['results']:
        if validar == 0:
            writer.writerow(dict(date=datetime.today().strftime('%x %X'),
                                 license_plate=result['plate'],
                                 score=result['score'],
                                 dscore=result['dscore'],
                                 ))
        # if validar==1:
        #    messagebox.showerror("ERROR", 'PLACA REPETIDA')
        # if validar==2:
        #    messagebox.showerror("ERROR", 'MAS de ' + str(max_num_plate)+ ' AUTOS')


def tablero(frame, lista_placas, fechas):
    pizarra = np.zeros((frame.shape[0], 300, 3), dtype=np.uint8)
    # titulo
    cv2.putText(pizarra, "Plates readed :", (10, 20),
                cv2.FONT_HERSHEY_COMPLEX, .6, (255, 0, 255), 1, cv2.LINE_AA)
    # imprimir placas en el tablero
    y = 25
    for placa, fecha in zip(lista_placas, fechas):
        y += 16
        cv2.putText(pizarra, placa, (10, y),
                    cv2.FONT_HERSHEY_COMPLEX, .5, (255, 254, 255), 1, cv2.LINE_AA)
        cv2.putText(
            pizarra, fecha[0], (90, y), cv2.FONT_HERSHEY_COMPLEX, .5, (255, 254, 255), 1, cv2.LINE_AA)
        cv2.putText(pizarra, fecha[1], (190, y),
                    cv2.FONT_HERSHEY_COMPLEX, .5, (255, 254, 255), 1, cv2.LINE_AA)
    # unir pizarra con imagen
    combinado = np.hstack([frame, pizarra])
    return combinado


async def verificar_imagen():
    global foto
    global lista_placas
    global file
    global writer
    global max_num_plate
    global fechas
    while True:
        if foto is not None:
            datos = await leer_placa(foto)
            if 'status_code' in datos and datos['status_code'] != 200 and 'detail' in datos:
                print('Error al leer placa: ' + datos['detail'])
            await validar_placa(datos, lista_placas, file, writer, max_num_plate, fechas)
        await asyncio.sleep(5)


"""
Demo program that read plate
"""

foto = None
writer = None

lista_placas = []
fechas = []
file = 'save.csv'  # nombre del archivo de datos previamente creado
max_num_plate = 10
imagen = None


async def main_loop():
    global foto
    global lista_placas
    global file
    global writer
    global max_num_plate
    global fechas
    sg.ChangeLookAndFeel('Black')
    # define the window layout
    layout = [[sg.Text('PLATE ', size=(40, 1), justification='center', font='Helvetica 20')],
              [sg.Image(filename='', key='image')],
              [sg.Button('Read plate', size=(10, 1), font='Helvetica 14'),
               sg.Button('Map', size=(10, 1), font='Any 14'),
               sg.Button('Exit', size=(10, 1), font='Helvetica 14'),]]
    # create the window and show it without the plot
    window = sg.Window('Reconocimiento ', layout,
                       location=(800, 400))
    ######## ciclo para que la ventana se ejecute constantemente #######
    # se bare el archivo para escritura
    with open(file, 'w') as output:
        # se definen los campos del fichero
        fields = ['date', 'license_plate', 'score', 'dscore']
        writer = csv.DictWriter(output, fieldnames=fields)
        writer.writeheader()
        # se hace un ciclo para que la ventana se mantenga constantemente
        # ---===--- Event LOOP Read and display frames, operate the GUI --- #
        cap = cv2.VideoCapture(0)
        recording = True
        while True:
            event, values = window.Read(timeout=20)
            if event == 'Exit' or event == sg.WIN_CLOSED or event == sg.WINDOW_CLOSED:
                cap.release()  # libera el objeto de video
                break
            # elif event == 'Read plate':
            elif True or event is None:
                ret, frame = cap.read()
                # Crear tablero de resultados
                frame_tablero = tablero(frame, lista_placas, fechas)
                imgbytes = cv2.imencode('.png', frame_tablero)[
                    1].tobytes()  # ditto
                # window.FindElement('image').Update(data=imgbytes)
                # update window with image
                window['image'].update(data=imgbytes)

                foto = "temp.jpg"  # nombre de la imagen temporal a guardar
                # se guarda la imagen capturada por el video
                cv2.imwrite(foto, frame)
                # # se llama a la funcion leer placa
                # data = await leer_placa(foto)
                # validar_placa(data,lista_placas,file,writer,max_num_plate, fechas) #recibe la fecha y hora
            elif event == 'Map':
                break
            await asyncio.sleep(0)
            # if recording:
            #    ret, frame = cap.read()
            #    #Crear tablero de resultados
            #    frame_tablero=tablero(frame, lista_placas,fechas)
            #    imgbytes=cv2.imencode('.png', frame_tablero)[1].tobytes() #ditto
            #    window.FindElement('image').Update(data=imgbytes)
        window.close()  # cerrar todo
    sys.exit(0)


async def main():
    task_main = asyncio.create_task(main_loop())
    task_verificar_imagen = asyncio.create_task(verificar_imagen())
    await asyncio.gather(task_main, task_verificar_imagen)

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    await asyncio.gather(main())


Error al registrar placa en el sistema Cannot connect to host localhost:8090 ssl:default [Connect call failed ('127.0.0.1', 8090)]
Error al registrar placa en el sistema Cannot connect to host localhost:8090 ssl:default [Connect call failed ('127.0.0.1', 8090)]
Error al registrar placa en el sistema Cannot connect to host localhost:8090 ssl:default [Connect call failed ('127.0.0.1', 8090)]
Error al registrar placa en el sistema Cannot connect to host localhost:8090 ssl:default [Connect call failed ('127.0.0.1', 8090)]
Error al registrar placa en el sistema Cannot connect to host localhost:8090 ssl:default [Connect call failed ('127.0.0.1', 8090)]
Error al registrar placa en el sistema Cannot connect to host localhost:8090 ssl:default [Connect call failed ('127.0.0.1', 8090)]
Error al registrar placa en el sistema Cannot connect to host localhost:8090 ssl:default [Connect call failed ('127.0.0.1', 8090)]
Error al registrar placa en el sistema Cannot connect to host localhost:8090 ssl:de

CancelledError: 